# Matching and Weighting 

In [50]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from causalml.match import NearestNeighborMatch

In [52]:
df = pd.read_csv('../data/matchandweight.csv')
df.head()
df.describe()
df['treatment'].value_counts() #Show how many people received the treatment vs did not

treatment
0    510
1    490
Name: count, dtype: int64

In [54]:
#Estimate treatment via OLS
X = df[['age','income','education_years','treatment']]
y = df['outcome']
X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                outcome   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     1338.
Date:                Wed, 05 Jun 2024   Prob (F-statistic):               0.00
Time:                        13:37:21   Log-Likelihood:                -10614.
No. Observations:                1000   AIC:                         2.124e+04
Df Residuals:                     995   BIC:                         2.126e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            1.737e+04   2598.695      6.683      0.000    1.23e+04    2.25e+04
age               152.8870     26.606      5.746      0.000     100.677     205.097
income              1.5056      0.021     72.018      0.000       1.465       1.547
education_years  1140.2030    158.825      7.179      0.000     828.533    1451.873
treatment        4840.9429    624.955      7.746      0.000    3614.562    6067.323
==============================================================================
Omnibus:                        5.880   Durbin-Watson:                   2.050
Prob(Omnibus):                  0.053   Jarque-Bera (JB):                5.797
Skew:                          -0.184   Prob(JB):                       0.0551
Kurtosis:                       3.058   Cond. No.                     4.44e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.44e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [56]:
#Fit a logistic regression model with treatment as the dependent variable 
X = df[['age', 'income', 'education_years']]
y = df['treatment']
model = LogisticRegression()
model.fit(X, y)

LogisticRegression()

In [58]:
#Calculate the propensity score for (probability of) each individual receiving the treatment
df['propensity_score'] = model.predict_proba(X)[:, 1] #First column is p(0), second column (indexed as 1) is p(1)
df.head(10)
df.describe()

,age,income,education_years,treatment,outcome,propensity_score
count,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000
mean,50.231985,51062.543559,12.011668,0.49000,117992.509487,0.495998
std,11.750591,14961.815658,1.966909,0.50015,24876.431385,0.001172
min,11.104792,5894.170480,5.960976,0.00000,35467.343156,0.492329
25%,42.228916,40906.374665,10.704001,0.00000,102237.666738,0.495225
50%,50.303607,50946.156984,11.999498,0.00000,118294.809411,0.496007
75%,57.775327,60933.232652,13.321831,1.00000,134080.084351,0.496794
max,96.232778,97896.613518,19.852475,1.00000,191400.150580,0.499538


In [60]:
#Matching based on propensity score
nnm = NearestNeighborMatch(replace=False, ratio=1, random_state=1663) #Originally replace=True
matched_data = nnm.match(data=df, treatment_col='treatment', score_cols=['propensity_score'])
matched_data.head(10)
matched_data.describe()
matched_data['treatment'].value_counts()

treatment
1    468
0    468
Name: count, dtype: int64

In [106]:
y = matched_data['outcome']
X = matched_data[['age', 'income', 'education_years', 'treatment']]
X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                outcome   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     1164.
Date:                Wed, 05 Jun 2024   Prob (F-statistic):               0.00
Time:                        13:56:21   Log-Likelihood:                -9942.9
No. Observations:                 936   AIC:                         1.990e+04
Df Residuals:                     931   BIC:                         1.992e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            1.671e+04   2735.687      6.108      0.000    1.13e+04    2.21e+04
age               150.2000     27.928      5.378      0.000      95.390     205.010
income              1.5048      0.022     67.463      0.000       1.461       1.549
education_years  1197.3941    165.102      7.252      0.000     873.379    1521.410
treatment        4949.3368    651.391      7.598      0.000    3670.972    6227.702
==============================================================================
Omnibus:                        3.856   Durbin-Watson:                   2.098
Prob(Omnibus):                  0.145   Jarque-Bera (JB):                3.795
Skew:                          -0.156   Prob(JB):                        0.150
Kurtosis:                       3.022   Cond. No.                     4.48e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.48e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [76]:
#Calculating the Inverse Probability of Treatment Weight 
df['weight'] = np.where(df['treatment'] == 1, 1 / df['propensity_score'], 1 / (1 - df['propensity_score']))
df.head()

,age,income,education_years,treatment,outcome,propensity_score,weight
0,55.960570,70990.331549,10.649643,1,152339.853676,0.494437,2.022503
1,48.340828,63869.505244,11.710963,1,152799.125155,0.494995,2.020223
2,57.772262,50894.455549,10.415160,0,121284.340854,0.496012,1.984172
3,68.276358,40295.948334,11.384077,1,102945.315002,0.496842,2.012712
4,47.190160,60473.349704,8.212771,0,128573.308851,0.495261,1.981222


In [74]:
#Weighted Least Squares Regression
X = df[['age','income','education_years','treatment']]
X = sm.add_constant(X)
y = df['outcome']

wls_model = sm.WLS(y, X, weights=df['weight'])
wls_results = wls_model.fit()
wls_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                outcome   R-squared:                       0.843
Model:                            WLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     1339.
Date:                Wed, 05 Jun 2024   Prob (F-statistic):               0.00
Time:                        13:41:17   Log-Likelihood:                -10614.
No. Observations:                1000   AIC:                         2.124e+04
Df Residuals:                     995   BIC:                         2.126e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            1.735e+04   2598.129      6.679      0.000    1.23e+04    2.25e+04
age               152.8677     26.601      5.747      0.000     100.668     205.068
income              1.5055      0.021     72.040      0.000       1.465       1.547
education_years  1141.5267    158.799      7.189      0.000     829.908    1453.145
treatment        4840.9205    624.835      7.748      0.000    3614.774    6067.067
==============================================================================
Omnibus:                        5.895   Durbin-Watson:                   2.049
Prob(Omnibus):                  0.052   Jarque-Bera (JB):                5.814
Skew:                          -0.185   Prob(JB):                       0.0547
Kurtosis:                       3.057   Cond. No.                     4.44e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.44e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [108]:
#Using weights to estimate the Average Treatment Effect 
mean_outcome_treated = df[df['treatment'] == 1]['outcome'].mean()
mean_outcome_control = df[df['treatment'] == 0]['outcome'].mean()
print(mean_outcome_treated)
print(mean_outcome_control)
ate = mean_outcome_treated - mean_outcome_control
print(f"Estimated Average Treatment Effect: {ate}")

weighted_outcome_treated = np.average(df[df['treatment'] == 1]['outcome'], weights=df[df['treatment'] == 1]['weight'])
weighted_outcome_control = np.average(df[df['treatment'] == 0]['outcome'], weights=df[df['treatment'] == 0]['weight'])
print(weighted_outcome_treated)
print(weighted_outcome_control)

ate = weighted_outcome_treated - weighted_outcome_control
print(f"Estimated Average Treatment Effect: {ate}")

121404.82822908025
114714.0071656345
Estimated Average Treatment Effect: 6690.82106344575
121458.83881910935
114663.01249394713
Estimated Average Treatment Effect: 6795.826325162227
